In [38]:
from sqlalchemy import create_engine

# Import pandas, numpy, matplotlib, seaborn libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import datetime

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn import metrics

In [3]:
# URI = 'database-dublin-bus.cnhytboldqes.eu-west-1.rds.amazonaws.com'
# PORT = "3306"
# DB = "database-dublin-bus"
# USER = "group14"
# PASSWARD = "gRouP1422"

# engine = create_engine("mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWARD, URI, PORT, DB))

In [4]:
# df_GTFS = pd.read_sql_table("gtfsr", engine, parse_dates="True", index_col=0)

In [5]:
#df_GTFS.to_csv('GTFS.csv',index=False)

In [6]:
df_GTFS = pd.read_csv('GTFS_08_08.csv')

In [7]:
df_GTFS

,Time_of_db_update,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay
0,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,2,8220DB000341,120,120
1,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,3,8220B111761,60,60
2,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,4,8220B100421,180,180
3,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,5,8220B1021601,300,300
4,1658621100,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,20220724,10,8250DB000435,240,240
...,...,...,...,...,...,...,...,...,...
641153,1660212000,4021164.4.10-220-e20-1.613.O,10-220-e20-1,12:40:00,20220811,48,8380B246981,0,0
641154,1660212000,5673.23.10-603-wb2-1.25.O,10-603-wb2-1,12:40:00,20220811,12,8440B352481,0,0
641155,1660212000,4017466.4.10-4-e20-1.10.I,10-4-e20-1,12:45:00,20220811,11,8220B1353501,0,0
641156,1660212000,4021176.4.10-220-e20-1.612.O,10-220-e20-1,12:55:00,20220811,48,8380B246981,0,0


In [8]:
df_GTFS.loc[(df_GTFS['TripId'] == 'ga2-irl-02751-R-.65535') & (df_GTFS['StartDate'] == 20220802)]

,Time_of_db_update,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay
318648,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,2,8250DB003563,180,180
318649,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,3,8250DB000444,180,180
318650,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,4,8250DB000445,180,180
318651,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,5,8250DB004779,180,180
318652,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,6,8250DB007057,180,180
...,...,...,...,...,...,...,...,...,...
318711,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,65,8230DB004435,180,180
318712,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,66,8230DB004436,180,180
318713,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,67,8230DB005008,180,180
318714,1659463201,ga2-irl-02751-R-.65535,ga2-irl-02751-R-,18:00:00,20220802,68,8230DB004646,180,180


In [9]:
df_GTFS.dtypes

Time_of_db_update     int64
TripId               object
RouteId              object
StartTime            object
StartDate             int64
StopSequence          int64
StopId               object
Arr_delay             int64
Dep_delay             int64
dtype: object

In [10]:
#Classify the columns type
categorical_columns = df_GTFS[['TripId','RouteId','StopId']].columns
continuous_columns = df_GTFS[['Arr_delay','Dep_delay','StopSequence']].columns


for column in categorical_columns:
    df_GTFS[column] = df_GTFS[column].astype('category')  
    
for column in continuous_columns:
    df_GTFS[column] = df_GTFS[column].astype('int64')
    
df_GTFS['StartDate'] = df_GTFS['StartDate'].astype('str')
df_GTFS['StartTime'] = df_GTFS['StartTime'].astype('str')

In [11]:
df_GTFS['StartDate']=df_GTFS['StartDate'].str[:4] + '-' + df_GTFS['StartDate'].str[4:]
df_GTFS['StartDate']=df_GTFS['StartDate'].str[:7] + '-' + df_GTFS['StartDate'].str[7:]

In [12]:
#df_GTFS['StartDate'] = df_GTFS['StartDate'].astype('str')

In [13]:
#Convert the 'StartDate' into datetime
#df_GTFS['StartDate'] = pd.to_datetime(df_GTFS[column]).astype('datetime64')
#Concatenate the date with the time

df_GTFS['StartDate'] = df_GTFS['StartDate'] + str(' ') + df_GTFS['StartTime']

In [14]:
df_GTFS['StartDate'] = pd.to_datetime(df_GTFS['StartDate']).astype('datetime64')

In [15]:
df_GTFS.dtypes

Time_of_db_update             int64
TripId                     category
RouteId                    category
StartTime                    object
StartDate            datetime64[ns]
StopSequence                  int64
StopId                     category
Arr_delay                     int64
Dep_delay                     int64
dtype: object

In [16]:
df_GTFS = df_GTFS.drop(columns = ['Time_of_db_update'])

In [17]:
# counting the null values.
df_GTFS.isna().sum()

TripId          0
RouteId         0
StartTime       0
StartDate       0
StopSequence    0
StopId          0
Arr_delay       0
Dep_delay       0
dtype: int64

In [18]:
df_GTFS['TripId'].value_counts()

4040812.7.10-51-e20-1.114.I      283
ga2-irl-02751-R-.65535           248
4041609.7.10-22-e20-1.45.O       243
3991.1.60-40-b12-1.219.I         240
4873.1.60-40-b12-1.219.I         220
                                ... 
4031916.5.10-405-e20-1.1510.O      1
4031921.5.10-405-e20-1.1508.I      1
4031948.5.10-407-e20-1.1514.O      1
10618.3.60-123-d12-1.74.I          1
4026237.5.10-704-e20-1.1964.I      1
Name: TripId, Length: 42606, dtype: int64

In [19]:
df_GTFS['RouteId'].value_counts()

60-40-b12-1     19780
60-13-b12-1     17267
10-220-e20-1    17014
60-N4-b12-1     12768
60-15-b12-1     10815
                ...  
10-259-e20-1        2
10-467-e20-1        2
10-495-e20-1        2
10-461-e20-1        1
10-378-e20-1        1
Name: RouteId, Length: 469, dtype: int64

In [20]:
df_GTFS

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay
0,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,2,8220DB000341,120,120
1,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,3,8220B111761,60,60
2,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,4,8220B100421,180,180
3,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,5,8220B1021601,300,300
4,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,10,8250DB000435,240,240
...,...,...,...,...,...,...,...,...
641153,4021164.4.10-220-e20-1.613.O,10-220-e20-1,12:40:00,2022-08-11 12:40:00,48,8380B246981,0,0
641154,5673.23.10-603-wb2-1.25.O,10-603-wb2-1,12:40:00,2022-08-11 12:40:00,12,8440B352481,0,0
641155,4017466.4.10-4-e20-1.10.I,10-4-e20-1,12:45:00,2022-08-11 12:45:00,11,8220B1353501,0,0
641156,4021176.4.10-220-e20-1.612.O,10-220-e20-1,12:55:00,2022-08-11 12:55:00,48,8380B246981,0,0


In [21]:
df_GTFS['RouteId'].str.split('-')[1][1]

'133'

In [22]:
df_temporary = df_GTFS['RouteId'].str.split('-',n=2,expand=True)
df_temporary

,0,1,2
0,10,133,e20-1
1,10,133,e20-1
2,10,133,e20-1
3,10,133,e20-1
4,10,133,e20-1
...,...,...,...
641153,10,220,e20-1
641154,10,603,wb2-1
641155,10,4,e20-1
641156,10,220,e20-1


In [23]:
df_temporary[1]

0         133
1         133
2         133
3         133
4         133
         ... 
641153    220
641154    603
641155      4
641156    220
641157    205
Name: 1, Length: 641158, dtype: object

In [24]:
df_GTFS['LineId'] = df_temporary[1]

In [25]:
df_GTFS

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
0,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,2,8220DB000341,120,120,133
1,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,3,8220B111761,60,60,133
2,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,4,8220B100421,180,180,133
3,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,5,8220B1021601,300,300,133
4,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,10,8250DB000435,240,240,133
...,...,...,...,...,...,...,...,...,...
641153,4021164.4.10-220-e20-1.613.O,10-220-e20-1,12:40:00,2022-08-11 12:40:00,48,8380B246981,0,0,220
641154,5673.23.10-603-wb2-1.25.O,10-603-wb2-1,12:40:00,2022-08-11 12:40:00,12,8440B352481,0,0,603
641155,4017466.4.10-4-e20-1.10.I,10-4-e20-1,12:45:00,2022-08-11 12:45:00,11,8220B1353501,0,0,4
641156,4021176.4.10-220-e20-1.612.O,10-220-e20-1,12:55:00,2022-08-11 12:55:00,48,8380B246981,0,0,220


In [26]:
df_GTFS['LineId'].unique()

array(['133', '15', '740', '2', 'C6', '126', '722', 'C2', '220', '65',
       'N4', '101', '39A', '704', '331', '41', '103', 'C5', 'C1', '23',
       '22', '115', '32', '700', 'C3', '226', '769', '456', '52', '422',
       '225', '64', '304', '742', '111', '343', 'H1', 'H2', '40E', '51',
       '446', '120', '33', '350', '155', '16', '27', '13', '77A', '145',
       '182', '39', '109', '7A', 'C4', '336', '83', '40', '123', '401',
       '223', '122', '46A', '70', '150', '151', '38A', '65B', '424',
       '202', '7', '208', '105', '14', '4', '451', '536', '360', '215',
       '38', '18', '27A', '214', '55', '43', '404', '102', '405', '26',
       '79', 'L53', 'H3', '6', '717', '245', '15A', '423', '407', '15B',
       '270', '132', '175', '63', '714', '40D', '323', '37', '44', '605',
       '602', '601', '719', '303', '603', '59', '440', '84', '9', '237',
       '190', '11', '47', '140', '49', '282', '207', '216', '83A', '79A',
       '174', '355', '205', '17', '56A', '76', '41C', '203'

In [27]:
df_GTFS['LineId'].value_counts()

40     35698
13     29750
N4     20797
15     17406
220    17303
       ...  
259        2
467        2
495        2
378        1
461        1
Name: LineId, Length: 320, dtype: int64

In [28]:
df_GTFS[df_GTFS['LineId']=='77A']

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
1588,15429.2.60-77A-d12-1.56.I,60-77A-d12-1,08:00:00,2022-07-24 08:00:00,6,8230DB002590,60,60,77A
1589,15429.2.60-77A-d12-1.56.I,60-77A-d12-1,08:00:00,2022-07-24 08:00:00,9,8230DB002357,0,0,77A
1590,15429.2.60-77A-d12-1.56.I,60-77A-d12-1,08:00:00,2022-07-24 08:00:00,10,8230DB002566,60,60,77A
1591,15429.2.60-77A-d12-1.56.I,60-77A-d12-1,08:00:00,2022-07-24 08:00:00,12,8230DB002568,0,0,77A
1592,15429.2.60-77A-d12-1.56.I,60-77A-d12-1,08:00:00,2022-07-24 08:00:00,15,8230DB002570,60,60,77A
...,...,...,...,...,...,...,...,...,...
639419,3172.1.60-77A-b12-1.56.I,60-77A-b12-1,10:20:00,2022-08-11 10:20:00,64,8220DB000340,-180,-180,77A
639420,3172.1.60-77A-b12-1.56.I,60-77A-b12-1,10:20:00,2022-08-11 10:20:00,67,8220DB000352,-120,-120,77A
639421,3172.1.60-77A-b12-1.56.I,60-77A-b12-1,10:20:00,2022-08-11 10:20:00,68,8220DB000353,-120,0,77A
640491,3181.1.60-77A-b12-1.56.I,60-77A-b12-1,10:40:00,2022-08-11 10:40:00,9,8230DB002357,60,60,77A


In [29]:
#Remove the data of LineId which are in the 42_route list
lineid_list_42routes = ['77A', '16', '40D', '142', '83', '46A', '49', '9', '40', '151', '13', '15B', '61', '140', '79A', '38A', '69', '54A', '150', '150', '56A', '120', '41C', '39X', '68A', '84X', '38', '33X', '26', '44B', '42D', '33D', '40B', '46E', '118', '51D', '15D', '38B', '7B', '41X', '69X', '70D', '77X']

In [30]:
df_GTFS[df_GTFS['LineId'].isin(lineid_list_42routes)].index

Int64Index([  1466,   1467,   1468,   1469,   1470,   1471,   1472,   1473,
              1474,   1536,
            ...
            641052, 641095, 641102, 641103, 641106, 641115, 641116, 641117,
            641118, 641147],
           dtype='int64', length=174570)

In [31]:
df_GTFS=df_GTFS.drop(index=(df_GTFS[df_GTFS['LineId'].isin(lineid_list_42routes)].index))

In [32]:
df_GTFS

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
0,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,2,8220DB000341,120,120,133
1,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,3,8220B111761,60,60,133
2,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,4,8220B100421,180,180,133
3,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,5,8220B1021601,300,300,133
4,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,10,8250DB000435,240,240,133
...,...,...,...,...,...,...,...,...,...
641153,4021164.4.10-220-e20-1.613.O,10-220-e20-1,12:40:00,2022-08-11 12:40:00,48,8380B246981,0,0,220
641154,5673.23.10-603-wb2-1.25.O,10-603-wb2-1,12:40:00,2022-08-11 12:40:00,12,8440B352481,0,0,603
641155,4017466.4.10-4-e20-1.10.I,10-4-e20-1,12:45:00,2022-08-11 12:45:00,11,8220B1353501,0,0,4
641156,4021176.4.10-220-e20-1.612.O,10-220-e20-1,12:55:00,2022-08-11 12:55:00,48,8380B246981,0,0,220


In [33]:
#Only keep the LineId where the number of value is greater than 1000
list = df_GTFS['LineId'].value_counts().loc[lambda x: x<=1000].index
df_GTFS_DROP = df_GTFS[df_GTFS['LineId'].isin(list)]

In [34]:
df_GTFS_DROP

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
32,13214.3.60-C6-d12-1.350.O,60-C6-d12-1,00:05:00,2022-07-24 00:05:00,57,8260DB003907,120,120,C6
33,13214.3.60-C6-d12-1.350.O,60-C6-d12-1,00:05:00,2022-07-24 00:05:00,65,8260DB003914,60,60,C6
135,10474.11346.2-331-gad-1.264.O,2-331-gad-1,00:30:00,2022-07-24 00:30:00,38,8240DB003775,120,180,331
164,13277.3.60-C5-d12-1.348.O,60-C5-d12-1,00:34:00,2022-07-24 00:34:00,3,8220DB000397,60,60,C5
165,13277.3.60-C5-d12-1.348.O,60-C5-d12-1,00:34:00,2022-07-24 00:34:00,7,8220DB007732,0,0,C5
...,...,...,...,...,...,...,...,...,...
641139,4019854.4.10-187-e20-1.467.O,10-187-e20-1,12:10:00,2022-08-11 12:10:00,8,8520B1014701,-1140,-1140,187
641140,4019854.4.10-187-e20-1.467.O,10-187-e20-1,12:10:00,2022-08-11 12:10:00,9,8520B1318401,-1320,-1320,187
641141,4019854.4.10-187-e20-1.467.O,10-187-e20-1,12:10:00,2022-08-11 12:10:00,10,8520B1351901,-1980,-1980,187
641142,4019854.4.10-187-e20-1.467.O,10-187-e20-1,12:10:00,2022-08-11 12:10:00,11,8520B1357201,-2640,-2640,187


In [35]:
df_GTFS_DROP['LineId'].unique()

array(['C6', '331', 'C5', '226', '422', '225', '40E', '446', '182', '336',
       '65B', '360', '27A', '102', 'L53', '717', '423', '407', '270',
       '132', '175', '63', '714', '323', '719', '59', '190', '282', '83A',
       '174', '355', '17', '420', 'L58', '100', '206', '604', '302',
       '333', '238', '168', '275', '756', '708', '185', '114', '75',
       '160', '279', '332', '767', '280', '716', '184', '27B', '727',
       '41B', '233', '770', '419', '480', '72', '450', '163', '107',
       '345', '472', '314', '366', '260', '252', '724', '728', '951',
       '478', '108', '718', '104', '745', '421', '341', '763', '239',
       '771', '761', '750', '757', '125', '721', '494', '240', 'X30',
       '774', '758', '162', '16D', 'X27', 'X25', '795', '161', '27X',
       '241', '84A', '212', '271', '313', '167', '53', '201', '425',
       '306', '213', '329', '166', '370', '466', '379', '257', '372',
       '276', '471', '444', '272', '243', '747', '173', '320', '328',
       '720', 

In [36]:
#Only keep the LineId where the number of value is greater than 1000
list = df_GTFS['LineId'].value_counts().loc[lambda x: x>1000].index
df_GTFS = df_GTFS[df_GTFS['LineId'].isin(list)]

In [37]:
df_GTFS['LineId'].unique()

array(['133', '15', '740', '2', '126', '722', 'C2', '220', '65', 'N4',
       '101', '39A', '704', '41', '103', 'C1', '23', '22', '115', '32',
       '700', 'C3', '769', '456', '52', '64', '304', '742', '111', '343',
       'H1', 'H2', '51', '33', '350', '155', '27', '145', '39', '109',
       '7A', 'C4', '123', '401', '223', '122', '70', '424', '202', '7',
       '208', '105', '14', '4', '451', '536', '215', '18', '214', '55',
       '43', '404', '405', '79', 'H3', '6', '245', '15A', '37', '44',
       '605', '602', '601', '303', '603', '440', '84', '237', '11', '47',
       '207', '216', '205', '76', '203', '301', 'irl', '409', '402',
       '130', '42', 'L54', '458', 'L59', '1', '236', '73', '952', '68',
       '760'], dtype=object)

In [33]:
df_GTFS

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
0,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,2,8220DB000341,120,120,133
1,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,3,8220B111761,60,60,133
2,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,4,8220B100421,180,180,133
3,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,5,8220B1021601,300,300,133
4,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,10,8250DB000435,240,240,133
...,...,...,...,...,...,...,...,...,...
385346,4022171.4.10-301-e20-1.1074.I,10-301-e20-1,14:35:00,2022-08-04 14:35:00,25,8410B607821,0,0,301
385347,4022356.4.10-220-e20-1.613.O,10-220-e20-1,14:40:00,2022-08-04 14:40:00,48,8380B246981,0,0,220
385348,4020771.4.10-205-e20-1.505.I,10-205-e20-1,14:40:00,2022-08-04 14:40:00,23,8370B2419901,0,0,205
385349,4021951.4.10-216-e20-1.592.I,10-216-e20-1,14:40:00,2022-08-04 14:40:00,28,8380B246981,0,0,216


In [34]:
df_GTFS['LineId'].value_counts()

N4     12633
15     10522
220    10181
27      9844
155     7471
       ...  
42      1155
1       1108
68      1085
245     1033
236     1006
Name: LineId, Length: 85, dtype: int64

In [35]:
df_GTFS['StopId'].value_counts()

8380B246981     1650
8370B2419901     960
8370B2414601     873
8220DB007392     693
8370B2420501     664
                ... 
8230DB002402       0
8230DB002401       0
8230DB002400       0
8230DB002399       0
8380B2342901       0
Name: StopId, Length: 8041, dtype: int64

In [36]:
df_GTFS[df_GTFS['Arr_delay'] != 0]

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
0,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,2,8220DB000341,120,120,133
1,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,3,8220B111761,60,60,133
2,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,4,8220B100421,180,180,133
3,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,5,8220B1021601,300,300,133
4,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,10,8250DB000435,240,240,133
...,...,...,...,...,...,...,...,...,...
385320,4020581.4.10-202-e20-1.482.O,10-202-e20-1,13:50:00,2022-08-04 13:50:00,21,8370B299001,120,0,202
385326,4021690.4.10-214-e20-1.565.O,10-214-e20-1,13:56:00,2022-08-04 13:56:00,18,8370B243491,180,0,214
385334,4018957.4.10-109-e20-1.276.O,10-109-e20-1,14:15:00,2022-08-04 14:15:00,2,8220B1358201,-7980,-7980,109
385335,4018957.4.10-109-e20-1.276.O,10-109-e20-1,14:15:00,2022-08-04 14:15:00,3,8220B1044901,-8040,-8040,109


In [37]:
df_GTFS[df_GTFS['Arr_delay'] < 0]

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
12,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,33,8220DB000301,-60,0,15
22,4035087.6.10-740-0-e20-1.2281.I,10-740-0-e20-1,00:00:00,2022-07-24 00:00:00,9,8220B111751,-60,-60,740
23,4035087.6.10-740-0-e20-1.2281.I,10-740-0-e20-1,00:00:00,2022-07-24 00:00:00,10,8220DB000277,-120,-60,740
24,4035087.6.10-740-0-e20-1.2281.I,10-740-0-e20-1,00:00:00,2022-07-24 00:00:00,11,8220B111771,-120,-120,740
25,4035087.6.10-740-0-e20-1.2281.I,10-740-0-e20-1,00:00:00,2022-07-24 00:00:00,12,8220B111761,-180,-180,740
...,...,...,...,...,...,...,...,...,...
385253,4020437.4.10-202-e20-1.482.O,10-202-e20-1,13:10:00,2022-08-04 13:10:00,20,8370B2368501,-1740,-1740,202
385254,4020437.4.10-202-e20-1.482.O,10-202-e20-1,13:10:00,2022-08-04 13:10:00,21,8370B299001,-1860,-2160,202
385334,4018957.4.10-109-e20-1.276.O,10-109-e20-1,14:15:00,2022-08-04 14:15:00,2,8220B1358201,-7980,-7980,109
385335,4018957.4.10-109-e20-1.276.O,10-109-e20-1,14:15:00,2022-08-04 14:15:00,3,8220B1044901,-8040,-8040,109


In [38]:
df_GTFS[df_GTFS['Arr_delay'] == 0]

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
7,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,16,8350DB006000,0,0,133
9,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,26,8350B3202601,0,0,133
13,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,34,8220DB004495,0,60,15
21,4035087.6.10-740-0-e20-1.2281.I,10-740-0-e20-1,00:00:00,2022-07-24 00:00:00,3,8310B136611,0,0,740
50,13076.3.60-C2-d12-1.341.O,60-C2-d12-1,00:14:00,2022-07-24 00:14:00,12,8220DB000396,0,0,C2
...,...,...,...,...,...,...,...,...,...
385346,4022171.4.10-301-e20-1.1074.I,10-301-e20-1,14:35:00,2022-08-04 14:35:00,25,8410B607821,0,0,301
385347,4022356.4.10-220-e20-1.613.O,10-220-e20-1,14:40:00,2022-08-04 14:40:00,48,8380B246981,0,0,220
385348,4020771.4.10-205-e20-1.505.I,10-205-e20-1,14:40:00,2022-08-04 14:40:00,23,8370B2419901,0,0,205
385349,4021951.4.10-216-e20-1.592.I,10-216-e20-1,14:40:00,2022-08-04 14:40:00,28,8380B246981,0,0,216


In [39]:
df_GTFS[df_GTFS['Arr_delay'] != df_GTFS['Dep_delay']]

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
12,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,33,8220DB000301,-60,0,15
13,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,34,8220DB004495,0,60,15
14,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,35,8220DB007581,60,120,15
16,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,38,8220DB001285,300,360,15
23,4035087.6.10-740-0-e20-1.2281.I,10-740-0-e20-1,00:00:00,2022-07-24 00:00:00,10,8220DB000277,-120,-60,740
...,...,...,...,...,...,...,...,...,...
385307,4020501.4.10-202-e20-1.479.I,10-202-e20-1,13:40:00,2022-08-04 13:40:00,22,8370B299021,240,60,202
385310,5732.23.10-603-wb2-1.25.O,10-603-wb2-1,13:40:00,2022-08-04 13:40:00,14,8440B352501,240,120,603
385317,4023348.4.10-304-e20-1.1118.O,10-304-e20-1,13:43:00,2022-08-04 13:43:00,19,8400B6078301,60,0,304
385320,4020581.4.10-202-e20-1.482.O,10-202-e20-1,13:50:00,2022-08-04 13:50:00,21,8370B299001,120,0,202


In [40]:
df_GTFS['RouteId'].value_counts()

10-220-e20-1    10025
60-N4-d12-1      8029
60-15-d12-1      6591
10-51-e20-1      6195
60-27-d12-1      6115
                ...  
2-238-gad-1         0
2-185-gad-1         0
2-184-gad-1         0
2-18-gad-1          0
10-100-e20-1        0
Name: RouteId, Length: 461, dtype: int64

In [41]:
#Now we investigate the info with ['TripId'] == '4040812.7.10-51-e20-1.114.I'

In [42]:
df_GTFS[df_GTFS['RouteId'] == '10-220-e20-1']

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
57,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,6,8380B245111,0,60,220
58,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,15,8380B246771,60,120,220
59,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,18,8380B243961,180,180,220
60,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,20,8380B245341,180,240,220
61,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,21,8380B245241,240,300,220
...,...,...,...,...,...,...,...,...,...
385288,4021141.4.10-220-e20-1.612.O,10-220-e20-1,13:25:00,2022-08-04 13:25:00,48,8380B246981,0,0,220
385305,4022057.4.10-220-e20-1.613.O,10-220-e20-1,13:40:00,2022-08-04 13:40:00,48,8380B246981,0,0,220
385323,4021084.4.10-220-e20-1.612.O,10-220-e20-1,13:55:00,2022-08-04 13:55:00,48,8380B246981,0,0,220
385332,4022340.4.10-220-e20-1.613.O,10-220-e20-1,14:10:00,2022-08-04 14:10:00,48,8380B246981,0,0,220


In [43]:
df_GTFS[(df_GTFS['RouteId'] == '10-220-e20-1')&(df_GTFS['StartDate'] == '2022-07-24 00:15:00')]

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
57,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,6,8380B245111,0,60,220
58,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,15,8380B246771,60,120,220
59,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,18,8380B243961,180,180,220
60,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,20,8380B245341,180,240,220
61,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,21,8380B245241,240,300,220
62,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,26,8380B234611,360,360,220
63,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,32,8370B2413801,420,420,220
64,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,40,8370B2414501,480,480,220
65,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,44,8370B2414601,480,360,220
66,4038474.6.10-220-e20-1.607.I,10-220-e20-1,00:15:00,2022-07-24 00:15:00,45,8370B244161,480,480,220


In [44]:
df_GTFS[(df_GTFS['RouteId'] == '10-220-e20-1')&(df_GTFS['StartDate'] == '2022-08-04 13:25:00')]

,TripId,RouteId,StartTime,StartDate,StopSequence,StopId,Arr_delay,Dep_delay,LineId
385287,4021141.4.10-220-e20-1.612.O,10-220-e20-1,13:25:00,2022-08-04 13:25:00,36,8370B2420501,120,0,220
385288,4021141.4.10-220-e20-1.612.O,10-220-e20-1,13:25:00,2022-08-04 13:25:00,48,8380B246981,0,0,220


In [94]:
df_stoptimes = pd.read_csv('../google_transit_combined/stop_times.txt')

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [95]:
df_stoptimes

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,8085.11324.2-952-ga2-1.346.I,06:25:00,06:25:00,8230DB006290,1,NaN,0,0,0.00
1,8085.11324.2-952-ga2-1.346.I,06:26:00,06:26:00,8230DB007800,2,NaN,0,0,3824.57
2,8085.11324.2-952-ga2-1.346.I,06:28:00,06:28:00,8230DB003890,3,NaN,0,0,4288.93
3,8085.11324.2-952-ga2-1.346.I,06:30:00,06:30:00,8230DB003891,4,NaN,0,0,4449.84
4,8085.11324.2-952-ga2-1.346.I,06:32:00,06:32:00,8230DB003949,5,NaN,0,0,5031.08
...,...,...,...,...,...,...,...,...,...
3865927,7133.2.60-X32-b12-1.118.I,08:44:51,08:44:51,8220DB002810,35,Earlsfort Terrace,1,0,20646.85
3865928,7133.2.60-X32-b12-1.118.I,08:46:19,08:46:19,8220DB002811,36,Earlsfort Terrace,1,0,20832.82
3865929,7133.2.60-X32-b12-1.118.I,08:49:19,08:49:19,8220DB000785,37,Earlsfort Terrace,1,0,21204.99
3865930,7133.2.60-X32-b12-1.118.I,08:53:19,08:53:19,8220DB000786,38,Earlsfort Terrace,1,0,21561.46


In [96]:
df_stoptimes[df_stoptimes['trip_id']=='3883541.10.10-2-e20-1.1.I']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
988725,3883541.10.10-2-e20-1.1.I,10:00:00,10:00:00,8340B3555101,1,NaN,0,0,0.00
988726,3883541.10.10-2-e20-1.1.I,10:15:00,10:15:00,8340B351131,2,NaN,0,0,12495.53
988727,3883541.10.10-2-e20-1.1.I,10:23:00,10:23:00,8340B3555201,3,NaN,0,0,21971.91
988728,3883541.10.10-2-e20-1.1.I,10:36:00,10:36:00,8340B355111,4,NaN,0,0,33753.83
988729,3883541.10.10-2-e20-1.1.I,10:42:00,10:42:00,8340B3551901,5,NaN,0,0,39580.03
988730,3883541.10.10-2-e20-1.1.I,10:48:00,10:48:00,8340B355231,6,NaN,0,0,46022.27
988731,3883541.10.10-2-e20-1.1.I,10:53:00,10:53:00,8340B355121,7,NaN,0,0,50963.62
988732,3883541.10.10-2-e20-1.1.I,11:09:00,11:09:00,8350B3551801,8,NaN,0,0,66827.57
988733,3883541.10.10-2-e20-1.1.I,11:11:00,11:11:00,8350B351481,9,NaN,0,0,68019.79
988734,3883541.10.10-2-e20-1.1.I,11:13:00,11:13:00,8350B1355301,10,NaN,0,0,68608.43


In [47]:
df_stoptimes[(df_stoptimes['trip_id'].str.contains('60-13-d12-1'))]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
1916870,7374.y1001.60-13-d12-1.18.O,11:15:00,11:15:00,8240DB006235,1,Grange Castle,0,0,0.00
1916871,7374.y1001.60-13-d12-1.18.O,11:17:15,11:17:15,8240DB000327,2,Grange Castle,0,0,1187.70
1916872,7374.y1001.60-13-d12-1.18.O,11:20:09,11:20:09,8220DB006016,3,Grange Castle,0,0,2729.39
1916873,7374.y1001.60-13-d12-1.18.O,11:21:37,11:21:37,8220DB006017,4,Grange Castle,0,0,3181.41
1916874,7374.y1001.60-13-d12-1.18.O,11:22:11,11:22:11,8220DB007192,5,Grange Castle,0,0,3358.81
...,...,...,...,...,...,...,...,...,...
1954998,14706.y1003.60-13-d12-1.30.O,22:38:20,22:38:20,8230DB002145,83,Grange Castle,0,0,28650.42
1954999,14706.y1003.60-13-d12-1.30.O,22:38:48,22:38:48,8230DB004670,84,Grange Castle,0,0,28916.89
1955000,14706.y1003.60-13-d12-1.30.O,22:39:12,22:39:12,8230DB002165,85,Grange Castle,0,0,29143.30
1955001,14706.y1003.60-13-d12-1.30.O,22:39:30,22:39:30,8230DB002166,86,Grange Castle,0,0,29306.35


In [48]:
df_stoptimes[(df_stoptimes['trip_id'].str.contains('60-13-d12-1')) & df_stoptimes['arrival_time'].str.contains('22:30:00')]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
1929998,7434.y1001.60-13-d12-1.20.O,22:30:00,22:30:00,8240DB006235,1,Grange Castle,0,0,0.00
1930694,1348.y1002.60-13-d12-1.20.O,22:30:00,22:30:00,8240DB006235,1,Grange Castle,0,0,0.00
1931999,14540.y1003.60-13-d12-1.20.O,22:30:00,22:30:00,8240DB006235,1,Grange Castle,0,0,0.00
1946843,7394.y1001.60-13-d12-1.23.I,22:30:00,22:30:00,8220DB001990,40,Harristown,0,0,15985.33
1947326,7441.y1001.60-13-d12-1.23.I,22:30:00,22:30:00,8230DB007229,1,Harristown,0,0,0.00
1949240,1408.y1002.60-13-d12-1.24.I,22:30:00,22:30:00,8230DB007229,1,Harristown,0,0,0.00
1950545,14691.y1003.60-13-d12-1.24.I,22:30:00,22:30:00,8230DB007229,1,Harristown,0,0,0.00


In [49]:
test_list = df_stoptimes['trip_id'][(df_stoptimes['trip_id'].str.contains('60-13-d12-1')) & df_stoptimes['arrival_time'].str.contains('22:30:00')].tolist()

In [50]:
df_stoptimes[df_stoptimes['trip_id'].isin(test_list)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
1929998,7434.y1001.60-13-d12-1.20.O,22:30:00,22:30:00,8240DB006235,1,Grange Castle,0,0,0.00
1929999,7434.y1001.60-13-d12-1.20.O,22:31:58,22:31:58,8240DB000327,2,Grange Castle,0,0,1187.70
1930000,7434.y1001.60-13-d12-1.20.O,22:34:29,22:34:29,8220DB006016,3,Grange Castle,0,0,2729.39
1930001,7434.y1001.60-13-d12-1.20.O,22:35:35,22:35:35,8220DB006017,4,Grange Castle,0,0,3181.41
1930002,7434.y1001.60-13-d12-1.20.O,22:36:01,22:36:01,8220DB007192,5,Grange Castle,0,0,3358.81
...,...,...,...,...,...,...,...,...,...
1950627,14691.y1003.60-13-d12-1.24.I,23:36:22,23:36:22,8220DB006238,83,Harristown,0,0,29681.71
1950628,14691.y1003.60-13-d12-1.24.I,23:36:49,23:36:49,8220DB006182,84,Harristown,0,0,29957.54
1950629,14691.y1003.60-13-d12-1.24.I,23:37:11,23:37:11,8220DB000322,85,Harristown,0,0,30180.39
1950630,14691.y1003.60-13-d12-1.24.I,23:38:51,23:38:51,8240DB000323,86,Harristown,0,0,31190.45


In [51]:
df_GTFS_routeid_list = df_GTFS['RouteId'].unique().tolist()

In [52]:
df_GTFS_routeid_list

['10-133-e20-1',
 '60-15-d12-1',
 '10-740-0-e20-1',
 '10-2-e20-1',
 '17-126-cm1-1',
 '10-722-2-e20-1',
 '60-C2-d12-1',
 '10-220-e20-1',
 '60-65-d12-1',
 '60-N4-d12-1',
 '10-101-e20-1',
 '60-39A-d12-1',
 '10-704-e20-1',
 '60-41-d12-1',
 '10-103-e20-1',
 '60-C1-d12-1',
 '10-23-e20-1',
 '10-22-e20-1',
 '10-115-e20-1',
 '10-32-e20-1',
 '10-700-e20-1',
 '60-C3-d12-1',
 '10-769-e20-1',
 '10-52-e20-1',
 '10-64-e20-1',
 '10-304-e20-1',
 '10-742-e20-1',
 '10-111-e20-1',
 '10-343-e20-1',
 '60-H1-d12-1',
 '60-H2-d12-1',
 '10-51-e20-1',
 '60-33-d12-1',
 '10-350-e20-1',
 '60-155-d12-1',
 '60-27-d12-1',
 '60-145-d12-1',
 '60-39-d12-1',
 '10-109-e20-1',
 '60-7A-d12-1',
 '60-C4-d12-1',
 '60-123-d12-1',
 '10-401-e20-1',
 '60-122-d12-1',
 '60-70-d12-1',
 '60-52-d12-1',
 '10-202-e20-1',
 '60-7-d12-1',
 '10-208-e20-1',
 '10-105-e20-1',
 '60-14-d12-1',
 '2-451-gad-1',
 '2-536-gad-1',
 '10-215-e20-1',
 '10-214-e20-1',
 '10-55-e20-1',
 '10-404-e20-1',
 '10-405-e20-1',
 '60-H3-d12-1',
 '60-6-d12-1',
 '10-245-

In [53]:
print(len(df_GTFS_routeid_list))

152


In [54]:
list_for_concat = []

counter = 0
for i in df_GTFS_routeid_list:
    list_for_concat.append(df_stoptimes[df_stoptimes['trip_id'].str.contains(i)])
    counter += 1
    print('counter = ' , counter, ', finish with ' , i)
    
df_stoptimes_new = pd.concat(list_for_concat)

counter =  1 , finish with  10-133-e20-1
counter =  2 , finish with  60-15-d12-1
counter =  3 , finish with  10-740-0-e20-1
counter =  4 , finish with  10-2-e20-1
counter =  5 , finish with  17-126-cm1-1
counter =  6 , finish with  10-722-2-e20-1
counter =  7 , finish with  60-C2-d12-1
counter =  8 , finish with  10-220-e20-1
counter =  9 , finish with  60-65-d12-1
counter =  10 , finish with  60-N4-d12-1
counter =  11 , finish with  10-101-e20-1
counter =  12 , finish with  60-39A-d12-1
counter =  13 , finish with  10-704-e20-1
counter =  14 , finish with  60-41-d12-1
counter =  15 , finish with  10-103-e20-1
counter =  16 , finish with  60-C1-d12-1
counter =  17 , finish with  10-23-e20-1
counter =  18 , finish with  10-22-e20-1
counter =  19 , finish with  10-115-e20-1
counter =  20 , finish with  10-32-e20-1
counter =  21 , finish with  10-700-e20-1
counter =  22 , finish with  60-C3-d12-1
counter =  23 , finish with  10-769-e20-1
counter =  24 , finish with  10-52-e20-1
counter = 

In [55]:
df_stoptimes_new

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
962881,3932872.70.10-133-e20-1.380.I,20:00:00,20:00:00,8350B1355601,1,NaN,0,0,0.00
962882,3932872.70.10-133-e20-1.380.I,20:02:00,20:02:00,8350B106131,2,NaN,0,0,701.08
962883,3932872.70.10-133-e20-1.380.I,20:03:00,20:03:00,8350B1060701,3,NaN,0,0,1144.17
962884,3932872.70.10-133-e20-1.380.I,20:08:00,20:08:00,8350B1355501,4,NaN,0,0,2577.73
962885,3932872.70.10-133-e20-1.380.I,20:10:00,20:10:00,8350B1060801,5,NaN,0,0,2971.39
...,...,...,...,...,...,...,...,...,...
3574976,4651.2.60-65-b12-1.282.I,16:38:52,16:38:52,8220DB001353,72,Poolbeg St,0,0,29853.30
3574977,4651.2.60-65-b12-1.282.I,16:41:41,16:41:41,8220DB001355,73,Poolbeg St,0,0,30312.35
3574978,4651.2.60-65-b12-1.282.I,16:43:19,16:43:19,8220DB001357,74,Poolbeg St,0,0,30603.53
3574979,4651.2.60-65-b12-1.282.I,16:44:42,16:44:42,8220DB001358,75,Poolbeg St,0,0,30885.55


In [58]:
df_stoptimes_new['trip_id_new'] = ''

In [59]:
counter = 0
for i in df_GTFS_routeid_list:
    df_stoptimes_new['trip_id_new'].loc[df_stoptimes_new['trip_id'].str.contains(i)] = i
    counter += 1
    print('counter = ' , counter, ', finish with ' , i)

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


counter =  1 , finish with  10-133-e20-1
counter =  2 , finish with  60-15-d12-1
counter =  3 , finish with  10-740-0-e20-1
counter =  4 , finish with  10-2-e20-1
counter =  5 , finish with  17-126-cm1-1
counter =  6 , finish with  10-722-2-e20-1
counter =  7 , finish with  60-C2-d12-1
counter =  8 , finish with  10-220-e20-1
counter =  9 , finish with  60-65-d12-1
counter =  10 , finish with  60-N4-d12-1
counter =  11 , finish with  10-101-e20-1
counter =  12 , finish with  60-39A-d12-1
counter =  13 , finish with  10-704-e20-1
counter =  14 , finish with  60-41-d12-1
counter =  15 , finish with  10-103-e20-1
counter =  16 , finish with  60-C1-d12-1
counter =  17 , finish with  10-23-e20-1
counter =  18 , finish with  10-22-e20-1
counter =  19 , finish with  10-115-e20-1
counter =  20 , finish with  10-32-e20-1
counter =  21 , finish with  10-700-e20-1
counter =  22 , finish with  60-C3-d12-1
counter =  23 , finish with  10-769-e20-1
counter =  24 , finish with  10-52-e20-1
counter = 

In [77]:
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('24','00')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('25','01')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('26','02')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('27','03')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('28','04')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('29','05')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('30','06')
df_stoptimes_new['arrival_time']=df_stoptimes_new['arrival_time'].str.replace('31','07')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('24','00')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('25','01')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('26','02')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('27','03')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('28','04')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('29','05')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('30','06')
df_stoptimes_new['departure_time']=df_stoptimes_new['departure_time'].str.replace('31','07')

In [78]:
df_stoptimes_new

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new
962881,3932872.70.10-133-e20-1.380.I,20:00:00,20:00:00,8350B1355601,1,NaN,0,0,0.00,10-133-e20-1
962882,3932872.70.10-133-e20-1.380.I,20:02:00,20:02:00,8350B106131,2,NaN,0,0,701.08,10-133-e20-1
962883,3932872.70.10-133-e20-1.380.I,20:03:00,20:03:00,8350B1060701,3,NaN,0,0,1144.17,10-133-e20-1
962884,3932872.70.10-133-e20-1.380.I,20:08:00,20:08:00,8350B1355501,4,NaN,0,0,2577.73,10-133-e20-1
962885,3932872.70.10-133-e20-1.380.I,20:10:00,20:10:00,8350B1060801,5,NaN,0,0,2971.39,10-133-e20-1
...,...,...,...,...,...,...,...,...,...,...
3574976,4651.2.60-65-b12-1.282.I,16:38:52,16:38:52,8220DB001353,72,Poolbeg St,0,0,29853.30,60-65-b12-1
3574977,4651.2.60-65-b12-1.282.I,16:41:41,16:41:41,8220DB001355,73,Poolbeg St,0,0,30312.35,60-65-b12-1
3574978,4651.2.60-65-b12-1.282.I,16:43:19,16:43:19,8220DB001357,74,Poolbeg St,0,0,30603.53,60-65-b12-1
3574979,4651.2.60-65-b12-1.282.I,16:44:42,16:44:42,8220DB001358,75,Poolbeg St,0,0,30885.55,60-65-b12-1


In [218]:
df_GTFS = df_GTFS.rename(columns={'RouteId':'trip_id_new','StartTime':'departure_time','StopSequence':'stop_sequence','StopId':'stop_id'})
df_GTFS

,TripId,trip_id_new,departure_time,StartDate,stop_sequence,stop_id,Arr_delay,Dep_delay,LineId
0,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,2,8220DB000341,120,120,133
1,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,3,8220B111761,60,60,133
2,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,4,8220B100421,180,180,133
3,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,5,8220B1021601,300,300,133
4,4035977.6.10-133-e20-1.360.O,10-133-e20-1,00:00:00,2022-07-24 00:00:00,10,8250DB000435,240,240,133
...,...,...,...,...,...,...,...,...,...
385346,4022171.4.10-301-e20-1.1074.I,10-301-e20-1,14:35:00,2022-08-04 14:35:00,25,8410B607821,0,0,301
385347,4022356.4.10-220-e20-1.613.O,10-220-e20-1,14:40:00,2022-08-04 14:40:00,48,8380B246981,0,0,220
385348,4020771.4.10-205-e20-1.505.I,10-205-e20-1,14:40:00,2022-08-04 14:40:00,23,8370B2419901,0,0,205
385349,4021951.4.10-216-e20-1.592.I,10-216-e20-1,14:40:00,2022-08-04 14:40:00,28,8380B246981,0,0,216


In [80]:
df_stoptimes_new[(df_stoptimes_new['trip_id_new']=='10-133-e20-1')&(df_stoptimes_new['stop_sequence']==1)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new
962881,3932872.70.10-133-e20-1.380.I,20:00:00,20:00:00,8350B1355601,1,NaN,0,0,0.0,10-133-e20-1
962914,3932884.70.10-133-e20-1.380.I,09:00:00,09:00:00,8350B1355601,1,NaN,0,0,0.0,10-133-e20-1
962947,3932888.70.10-133-e20-1.380.I,19:00:00,19:00:00,8350B1355601,1,NaN,0,0,0.0,10-133-e20-1
962980,3932855.70.10-133-e20-1.380.I,11:00:00,11:00:00,8350B1355601,1,NaN,0,0,0.0,10-133-e20-1
963013,3932865.70.10-133-e20-1.380.I,12:00:00,12:00:00,8350B1355601,1,NaN,0,0,0.0,10-133-e20-1
...,...,...,...,...,...,...,...,...,...,...
972295,3885548.10.10-133-e20-1.385.O,17:45:00,17:45:00,8220B1350001,1,NaN,0,0,0.0,10-133-e20-1
972301,3893851.20.10-133-e20-1.385.O,17:45:00,17:45:00,8220B1350001,1,NaN,0,0,0.0,10-133-e20-1
972307,3901287.30.10-133-e20-1.385.O,17:45:00,17:45:00,8220B1350001,1,NaN,0,0,0.0,10-133-e20-1
972313,3910374.40.10-133-e20-1.385.O,17:45:00,17:45:00,8220B1350001,1,NaN,0,0,0.0,10-133-e20-1


In [81]:
df_GTFS[df_GTFS['LineId']=='15']

,TripId,trip_id_new,departure_time,StartDate,stop_sequence,StopId,Arr_delay,Dep_delay,LineId
10,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,2,8220DB006319,120,120,15
11,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,32,8220DB004415,60,60,15
12,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,33,8220DB000301,-60,0,15
13,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,34,8220DB004495,0,60,15
14,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,35,8220DB007581,60,120,15
...,...,...,...,...,...,...,...,...,...
385065,169.1.60-15-b12-1.32.O,60-15-b12-1,12:50:00,2022-08-04 12:50:00,13,8220DB001274,0,0,15
385112,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,7,8230DB001151,-60,-60,15
385113,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,9,8230DB001153,0,0,15
385114,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,12,8230DB001155,-60,-60,15


In [73]:
df_GTFS['trip_id_new'].loc[df_GTFS['LineId']=='15'].unique()

['60-15-d12-1', '60-15-b12-1']
Categories (2, object): ['60-15-d12-1', '60-15-b12-1']

In [82]:
df_GTFS['departure_time'].loc[df_GTFS['trip_id_new']=='60-15-d12-1'].unique()

array(['00:00:00', '00:30:00', '01:00:00', '01:30:00', '02:00:00',
       '02:30:00', '04:30:00', '06:30:00', '08:00:00', '08:20:00',
       '08:30:00', '09:00:00', '09:20:00', '09:40:00', '10:00:00',
       '10:20:00', '10:40:00', '11:40:00', '12:00:00', '12:15:00',
       '12:30:00', '12:45:00', '13:30:00', '13:45:00', '14:00:00',
       '14:15:00', '14:30:00', '14:45:00', '15:45:00', '16:00:00',
       '16:30:00', '16:45:00', '17:30:00', '17:45:00', '18:00:00',
       '18:15:00', '18:30:00', '18:45:00', '19:45:00', '20:00:00',
       '20:15:00', '20:30:00', '20:45:00', '22:00:00', '22:15:00',
       '22:30:00', '22:45:00', '06:00:00', '06:10:00', '06:20:00',
       '06:40:00', '06:51:00', '07:34:00', '07:42:00', '07:45:00',
       '07:48:00', '07:50:00', '07:54:00', '07:58:00', '08:06:00',
       '08:12:00', '08:21:00', '08:22:00', '08:25:00', '08:42:00',
       '08:48:00', '09:50:00', '10:10:00', '10:30:00', '10:50:00',
       '11:50:00', '11:30:00', '12:10:00', '12:20:00', '12:40:

In [83]:
df_stoptimes_new['departure_time'].loc[(df_stoptimes_new['trip_id_new']=='60-15-d12-1')&(df_stoptimes_new['stop_sequence']==1)].unique()

array(['10:00:00', '10:06:00', '09:06:00', '10:15:00', '10:45:00',
       '11:00:00', '09:45:00', '11:15:00', '06:15:00', '05:06:00',
       '06:00:00', '17:06:00', '17:45:00', '18:00:00', '18:15:00',
       '07:15:00', '06:45:00', '06:06:00', '07:00:00', '08:12:00',
       '07:36:00', '07:48:00', '07:54:00', '08:03:00', '07:42:00',
       '08:21:00', '08:01:00', '07:45:00', '15:00:00', '15:10:00',
       '14:06:00', '14:40:00', '14:50:00', '15:20:00', '09:20:00',
       '08:40:00', '09:00:00', '22:15:00', '23:06:00', '21:15:00',
       '20:06:00', '00:00:00', '20:45:00', '23:45:00', '21:00:00',
       '22:45:00', '21:06:00', '22:00:00', '23:15:00', '23:00:00',
       '21:45:00', '22:06:00', '18:10:00', '18:20:00', '17:50:00',
       '17:40:00', '07:06:00', '08:00:00', '08:20:00', '16:45:00',
       '17:15:00', '17:00:00', '16:06:00', '19:15:00', '20:00:00',
       '20:15:00', '18:45:00', '19:00:00', '19:06:00', '19:45:00',
       '18:06:00', '02:00:00', '02:06:00', '00:06:00', '01:00:

In [ ]:
#Now we first try to work in the line 15

In [87]:
#save the trip_id in a list
list_id15=df_GTFS['trip_id_new'].loc[df_GTFS['LineId']=='15'].unique().tolist()

In [219]:
df_GTFS_15 = df_GTFS[df_GTFS['trip_id_new'].isin(list_id15)]
df_GTFS_15

,TripId,trip_id_new,departure_time,StartDate,stop_sequence,stop_id,Arr_delay,Dep_delay,LineId
10,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,2,8220DB006319,120,120,15
11,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,32,8220DB004415,60,60,15
12,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,33,8220DB000301,-60,0,15
13,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,34,8220DB004495,0,60,15
14,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,35,8220DB007581,60,120,15
...,...,...,...,...,...,...,...,...,...
385065,169.1.60-15-b12-1.32.O,60-15-b12-1,12:50:00,2022-08-04 12:50:00,13,8220DB001274,0,0,15
385112,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,7,8230DB001151,-60,-60,15
385113,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,9,8230DB001153,0,0,15
385114,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,12,8230DB001155,-60,-60,15


In [220]:
df_GTFS_15[df_GTFS_15.duplicated(keep=False)]

,TripId,trip_id_new,departure_time,StartDate,stop_sequence,stop_id,Arr_delay,Dep_delay,LineId
10,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,2,8220DB006319,120,120,15
11,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,32,8220DB004415,60,60,15
12,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,33,8220DB000301,-60,0,15
13,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,34,8220DB004495,0,60,15
14,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,35,8220DB007581,60,120,15
...,...,...,...,...,...,...,...,...,...
12638,13668.2.60-15-d12-1.32.O,60-15-d12-1,10:20:00,2022-07-24 10:20:00,24,8220DB000668,120,120,15
12639,13668.2.60-15-d12-1.32.O,60-15-d12-1,10:20:00,2022-07-24 10:20:00,28,8220DB000617,60,60,15
12640,13668.2.60-15-d12-1.32.O,60-15-d12-1,10:20:00,2022-07-24 10:20:00,32,8220DB004415,60,120,15
380041,128.1.60-15-b12-1.32.O,60-15-b12-1,11:50:00,2022-08-04 11:50:00,35,8220DB007581,0,0,15


In [221]:
#delete duplicated rows
df_GTFS_15 = df_GTFS_15.drop_duplicates()

In [222]:
#we save the unique departure time in df_GTFS_15
df_GTFS_15_deptime_list = df_GTFS_15['departure_time'].unique().tolist()
df_GTFS_15_deptime_list

['00:00:00',
 '00:30:00',
 '01:00:00',
 '01:30:00',
 '02:00:00',
 '02:30:00',
 '04:30:00',
 '06:30:00',
 '08:00:00',
 '08:20:00',
 '08:30:00',
 '09:00:00',
 '09:20:00',
 '09:40:00',
 '10:00:00',
 '10:20:00',
 '10:40:00',
 '11:40:00',
 '12:00:00',
 '12:15:00',
 '12:30:00',
 '12:45:00',
 '13:30:00',
 '13:45:00',
 '14:00:00',
 '14:15:00',
 '14:30:00',
 '14:45:00',
 '15:45:00',
 '16:00:00',
 '16:30:00',
 '16:45:00',
 '17:30:00',
 '17:45:00',
 '18:00:00',
 '18:15:00',
 '18:30:00',
 '18:45:00',
 '19:45:00',
 '20:00:00',
 '20:15:00',
 '20:30:00',
 '20:45:00',
 '22:00:00',
 '22:15:00',
 '22:30:00',
 '22:45:00',
 '06:00:00',
 '06:10:00',
 '06:20:00',
 '06:40:00',
 '06:51:00',
 '07:34:00',
 '07:42:00',
 '07:45:00',
 '07:48:00',
 '07:50:00',
 '07:54:00',
 '07:58:00',
 '08:06:00',
 '08:12:00',
 '08:21:00',
 '08:22:00',
 '08:25:00',
 '08:42:00',
 '08:48:00',
 '09:50:00',
 '10:10:00',
 '10:30:00',
 '10:50:00',
 '11:50:00',
 '11:30:00',
 '12:10:00',
 '12:20:00',
 '12:40:00',
 '12:50:00',
 '13:40:00',

In [223]:
df_stoptime_15 = df_stoptimes_new[df_stoptimes_new['trip_id_new'].isin(list_id15)]
df_stoptime_15

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new
2042230,7852.y1001.60-15-d12-1.31.I,10:00:00,10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1
2042231,7852.y1001.60-15-d12-1.31.I,10:00:33,10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1
2042232,7852.y1001.60-15-d12-1.31.I,10:02:03,10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1
2042233,7852.y1001.60-15-d12-1.31.I,10:03:06,10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1
2042234,7852.y1001.60-15-d12-1.31.I,10:05:00,10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1
...,...,...,...,...,...,...,...,...,...,...
3083778,4594.2.60-15-b12-1.33.O,17:49:54,17:49:54,8230DB004869,33,Ballycullen Road,0,0,9825.98,60-15-b12-1
3083779,4594.2.60-15-b12-1.33.O,17:53:14,17:53:14,8230DB004870,34,Ballycullen Road,0,0,10637.62,60-15-b12-1
3083780,4594.2.60-15-b12-1.33.O,17:53:39,17:53:39,8230DB003007,35,Ballycullen Road,0,0,10773.68,60-15-b12-1
3083781,4594.2.60-15-b12-1.33.O,17:54:58,17:54:58,8230DB006283,36,Ballycullen Road,0,0,11207.59,60-15-b12-1


In [383]:
#we will save the trip_id for the departure_time matches in the df_GTFS_15_deptime_list
trip_id_list = df_stoptime_15['trip_id'].loc[df_stoptime_15['departure_time'].isin(df_GTFS_15_deptime_list)].tolist()
print(len(trip_id_list))
print(trip_id_list)

1174
['7852.y1001.60-15-d12-1.31.I', '8042.y1001.60-15-d12-1.31.I', '8042.y1001.60-15-d12-1.31.I', '8055.y1001.60-15-d12-1.31.I', '8055.y1001.60-15-d12-1.31.I', '8263.y1001.60-15-d12-1.31.I', '8263.y1001.60-15-d12-1.31.I', '7837.y1001.60-15-d12-1.31.I', '8249.y1001.60-15-d12-1.31.I', '8059.y1001.60-15-d12-1.31.I', '8245.y1001.60-15-d12-1.31.I', '8286.y1001.60-15-d12-1.31.I', '8014.y1001.60-15-d12-1.31.I', '106.y1002.60-15-d12-1.31.I', '176.y1002.60-15-d12-1.31.I', '183.y1002.60-15-d12-1.31.I', '188.y1002.60-15-d12-1.31.I', '1654.y1002.60-15-d12-1.31.I', '1736.y1002.60-15-d12-1.31.I', '4595.y1002.60-15-d12-1.31.I', '141.y1002.60-15-d12-1.31.I', '180.y1002.60-15-d12-1.31.I', '192.y1002.60-15-d12-1.31.I', '202.y1002.60-15-d12-1.31.I', '13600.y1003.60-15-d12-1.31.I', '13600.y1003.60-15-d12-1.31.I', '13662.y1003.60-15-d12-1.31.I', '13662.y1003.60-15-d12-1.31.I', '13672.y1003.60-15-d12-1.31.I', '13672.y1003.60-15-d12-1.31.I', '7847.y1001.60-15-d12-1.31.I', '7875.y1001.60-15-d12-1.31.I', '801

In [384]:
df_stoptime_15['StartDate'] = ''
df_stoptime_15

<ipython-input-384-dcfc23f1c695>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stoptime_15['StartDate'] = ''


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate
2042230,7852.y1001.60-15-d12-1.31.I,10:00:00,10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1,
2042231,7852.y1001.60-15-d12-1.31.I,10:00:33,10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1,
2042232,7852.y1001.60-15-d12-1.31.I,10:02:03,10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1,
2042233,7852.y1001.60-15-d12-1.31.I,10:03:06,10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1,
2042234,7852.y1001.60-15-d12-1.31.I,10:05:00,10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1,
...,...,...,...,...,...,...,...,...,...,...,...
3083778,4594.2.60-15-b12-1.33.O,17:49:54,17:49:54,8230DB004869,33,Ballycullen Road,0,0,9825.98,60-15-b12-1,
3083779,4594.2.60-15-b12-1.33.O,17:53:14,17:53:14,8230DB004870,34,Ballycullen Road,0,0,10637.62,60-15-b12-1,
3083780,4594.2.60-15-b12-1.33.O,17:53:39,17:53:39,8230DB003007,35,Ballycullen Road,0,0,10773.68,60-15-b12-1,
3083781,4594.2.60-15-b12-1.33.O,17:54:58,17:54:58,8230DB006283,36,Ballycullen Road,0,0,11207.59,60-15-b12-1,


In [385]:
df_GTFS_15.dtypes

TripId            category
trip_id_new       category
departure_time      object
StartDate           object
stop_sequence        int64
stop_id           category
Arr_delay            int64
Dep_delay            int64
LineId              object
dtype: object

In [386]:
df_GTFS_15['StartDate']=df_GTFS_15['StartDate'].astype(str)

<ipython-input-386-29575f212a90>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_GTFS_15['StartDate']=df_GTFS_15['StartDate'].astype(str)


In [387]:
df_GTFS_15

,TripId,trip_id_new,departure_time,StartDate,stop_sequence,stop_id,Arr_delay,Dep_delay,LineId
10,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,2,8220DB006319,120,120,15
11,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,32,8220DB004415,60,60,15
12,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,33,8220DB000301,-60,0,15
13,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,34,8220DB004495,0,60,15
14,8292.3.60-15-d12-1.32.O,60-15-d12-1,00:00:00,2022-07-24 00:00:00,35,8220DB007581,60,120,15
...,...,...,...,...,...,...,...,...,...
385065,169.1.60-15-b12-1.32.O,60-15-b12-1,12:50:00,2022-08-04 12:50:00,13,8220DB001274,0,0,15
385112,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,7,8230DB001151,-60,-60,15
385113,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,9,8230DB001153,0,0,15
385114,1198.1.60-15-b12-1.31.I,60-15-b12-1,12:50:00,2022-08-04 12:50:00,12,8230DB001155,-60,-60,15


In [388]:
#we add merge StartDate to the 'df_stoptime_15'
# trip_id_list = ['7852.y1001.60-15-d12-1.31.I']
list_for_concat = []
for i in trip_id_list:
    start_timevalue = df_stoptime_15['departure_time'].loc[(df_stoptime_15['trip_id']==i)&(df_stoptime_15['stop_sequence']==1)].values[0]
    trip_id_new_value = df_stoptime_15['trip_id_new'].loc[(df_stoptime_15['trip_id']==i)&(df_stoptime_15['stop_sequence']==1)].values[0]
    startdate_list = df_GTFS_15['StartDate'].loc[(df_GTFS_15['departure_time']==start_timevalue)&(df_GTFS_15['trip_id_new']==trip_id_new_value)].unique().tolist()
    #print(startdate_list)
    
    for j in startdate_list:
        df_stoptime_15['StartDate'].loc[(df_stoptime_15['trip_id']==i)] = j
        list_for_concat.append(df_stoptime_15.loc[df_stoptime_15['StartDate']==j])
        
df_stoptime_15_new = pd.concat(list_for_concat)

['2022-07-24 10:00:00', '2022-07-25 10:00:00', '2022-07-26 10:00:00', '2022-07-27 10:00:00', '2022-07-28 10:00:00', '2022-07-29 10:00:00', '2022-07-30 10:00:00']
['2022-07-30 10:15:00']
['2022-07-30 10:15:00']
['2022-07-30 10:45:00']
['2022-07-30 10:45:00']
['2022-07-30 09:45:00']
['2022-07-30 09:45:00']
['2022-07-30 06:15:00']
['2022-07-25 06:00:00', '2022-07-26 06:00:00', '2022-07-27 06:00:00', '2022-07-28 06:00:00', '2022-07-29 06:00:00']


/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

['2022-07-24 17:45:00', '2022-07-30 17:45:00']
['2022-07-24 18:00:00', '2022-07-25 18:00:00', '2022-07-26 18:00:00', '2022-07-27 18:00:00', '2022-07-28 18:00:00', '2022-07-29 18:00:00', '2022-07-30 18:00:00']
['2022-07-24 18:15:00', '2022-07-30 18:15:00']
['2022-07-30 06:45:00']
['2022-07-25 08:12:00', '2022-07-26 08:12:00', '2022-07-27 08:12:00', '2022-07-28 08:12:00', '2022-07-29 08:12:00']
['2022-07-25 07:48:00', '2022-07-26 07:48:00', '2022-07-27 07:48:00', '2022-07-28 07:48:00', '2022-07-29 07:48:00']


/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

['2022-07-25 07:54:00', '2022-07-26 07:54:00', '2022-07-27 07:54:00', '2022-07-28 07:54:00', '2022-07-29 07:54:00']
['2022-07-26 08:03:00', '2022-07-27 08:03:00', '2022-07-28 08:03:00']
['2022-07-25 07:42:00', '2022-07-26 07:42:00', '2022-07-27 07:42:00', '2022-07-28 07:42:00', '2022-07-29 07:42:00']
['2022-07-25 08:21:00', '2022-07-26 08:21:00', '2022-07-27 08:21:00', '2022-07-29 08:21:00']
['2022-07-25 07:45:00']
['2022-07-25 15:10:00']
['2022-07-25 14:40:00', '2022-07-26 14:40:00', '2022-07-28 14:40:00', '2022-07-29 14:40:00']
['2022-07-25 14:50:00', '2022-07-26 14:50:00', '2022-07-28 14:50:00', '2022-07-29 14:50:00']

...

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

['2022-07-31 06:00:00', '2022-08-02 06:00:00', '2022-08-03 06:00:00', '2022-08-04 06:00:00']
['2022-07-31 06:00:00', '2022-08-02 06:00:00', '2022-08-03 06:00:00', '2022-08-04 06:00:00']
['2022-08-02 06:10:00', '2022-08-03 06:10:00', '2022-08-04 06:10:00']
['2022-08-02 06:10:00', '2022-08-03 06:10:00', '2022-08-04 06:10:00']
['2022-08-02 06:20:00', '2022-08-03 06:20:00', '2022-08-04 06:20:00']
['2022-08-02 06:20:00', '2022-08-03 06:20:00', '2022-08-04 06:20:00']


/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

['2022-08-02 14:40:00', '2022-08-03 14:40:00']
['2022-08-02 14:40:00', '2022-08-03 14:40:00']
['2022-08-02 14:50:00', '2022-08-03 14:50:00']
['2022-08-02 14:50:00', '2022-08-03 14:50:00']
[]
[]
[]
[]


/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [389]:
df_stoptime_15_new

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate
2042230,7852.y1001.60-15-d12-1.31.I,10:00:00,10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1,2022-07-24 10:00:00
2042231,7852.y1001.60-15-d12-1.31.I,10:00:33,10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1,2022-07-24 10:00:00
2042232,7852.y1001.60-15-d12-1.31.I,10:02:03,10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1,2022-07-24 10:00:00
2042233,7852.y1001.60-15-d12-1.31.I,10:03:06,10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1,2022-07-24 10:00:00
2042234,7852.y1001.60-15-d12-1.31.I,10:05:00,10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1,2022-07-24 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...
3083601,1154.2.60-15-b12-1.32.O,16:20:46,16:20:46,8230DB004869,66,Ballycullen Road,0,0,20871.25,60-15-b12-1,2022-08-03 14:50:00
3083602,1154.2.60-15-b12-1.32.O,16:01:07,16:01:07,8230DB004870,67,Ballycullen Road,0,0,21682.89,60-15-b12-1,2022-08-03 14:50:00
3083603,1154.2.60-15-b12-1.32.O,16:01:37,16:01:37,8230DB003007,68,Ballycullen Road,0,0,21818.96,60-15-b12-1,2022-08-03 14:50:00
3083604,1154.2.60-15-b12-1.32.O,16:03:14,16:03:14,8230DB006283,69,Ballycullen Road,0,0,22252.87,60-15-b12-1,2022-08-03 14:50:00


In [515]:
df_stoptimes_merged = pd.merge(df_stoptime_15_new,df_GTFS_15[['trip_id_new','StartDate','stop_sequence','stop_id','Arr_delay','Dep_delay','LineId']],how="left", on=['trip_id_new','StartDate','stop_sequence','stop_id'])
df_stoptimes_merged['Arr_delay'].fillna(0, inplace=True)
df_stoptimes_merged['Dep_delay'].fillna(0, inplace=True)
df_stoptimes_merged['LineId'].fillna(15, inplace=True)
df_stoptimes_merged

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate,Arr_delay,Dep_delay,LineId
0,7852.y1001.60-15-d12-1.31.I,10:00:00,10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
1,7852.y1001.60-15-d12-1.31.I,10:00:33,10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
2,7852.y1001.60-15-d12-1.31.I,10:02:03,10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
3,7852.y1001.60-15-d12-1.31.I,10:03:06,10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
4,7852.y1001.60-15-d12-1.31.I,10:05:00,10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335860,1154.2.60-15-b12-1.32.O,16:20:46,16:20:46,8230DB004869,66,Ballycullen Road,0,0,20871.25,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15
335861,1154.2.60-15-b12-1.32.O,16:01:07,16:01:07,8230DB004870,67,Ballycullen Road,0,0,21682.89,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15
335862,1154.2.60-15-b12-1.32.O,16:01:37,16:01:37,8230DB003007,68,Ballycullen Road,0,0,21818.96,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15
335863,1154.2.60-15-b12-1.32.O,16:03:14,16:03:14,8230DB006283,69,Ballycullen Road,0,0,22252.87,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15


In [516]:
df_stoptimes_merged.dtypes

trip_id                 object
arrival_time            object
departure_time          object
stop_id                 object
stop_sequence            int64
stop_headsign           object
pickup_type              int64
drop_off_type            int64
shape_dist_traveled    float64
trip_id_new             object
StartDate               object
Arr_delay              float64
Dep_delay              float64
LineId                  object
dtype: object

In [517]:
df_stoptimes_merged[df_stoptimes_merged['Arr_delay'] != 0]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate,Arr_delay,Dep_delay,LineId
19,7852.y1001.60-15-d12-1.31.I,10:20:55,10:20:55,8220DB001164,20,Clongriffin,0,0,6370.91,60-15-d12-1,2022-07-24 10:00:00,-60.0,-60.0,15
20,7852.y1001.60-15-d12-1.31.I,10:22:09,10:22:09,8220DB001165,21,Clongriffin,0,0,6742.12,60-15-d12-1,2022-07-24 10:00:00,-120.0,-120.0,15
21,7852.y1001.60-15-d12-1.31.I,10:23:23,10:23:23,8220DB001166,22,Clongriffin,0,0,7117.21,60-15-d12-1,2022-07-24 10:00:00,-60.0,-60.0,15
28,7852.y1001.60-15-d12-1.31.I,10:05:53,10:05:53,8220DB001071,29,Clongriffin,0,0,8757.90,60-15-d12-1,2022-07-24 10:00:00,-120.0,-120.0,15
32,7852.y1001.60-15-d12-1.31.I,10:34:46,10:34:46,8220DB001353,33,Clongriffin,0,0,9745.92,60-15-d12-1,2022-07-24 10:00:00,-60.0,-60.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335598,1154.2.60-15-b12-1.32.O,15:02:32,15:02:32,8220DB001273,12,Ballycullen Road,0,0,4239.62,60-15-b12-1,2022-08-03 14:50:00,60.0,60.0,15
335658,1154.2.60-15-b12-1.32.O,14:50:00,14:50:00,8220DB006319,2,Ballycullen Road,0,0,195.27,60-15-b12-1,2022-08-02 14:50:00,60.0,60.0,15
335660,1154.2.60-15-b12-1.32.O,14:52:34,14:52:34,8220DB006320,4,Ballycullen Road,0,0,902.08,60-15-b12-1,2022-08-02 14:50:00,120.0,120.0,15
335801,1154.2.60-15-b12-1.32.O,14:56:07,14:56:07,8220DB004596,7,Ballycullen Road,0,0,2381.37,60-15-b12-1,2022-08-03 14:50:00,60.0,60.0,15


In [518]:
df_stoptime_15[(df_stoptime_15['trip_id']=='7852.y1001.60-15-d12-1.31.I')]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate
2042230,7852.y1001.60-15-d12-1.31.I,10:00:00,10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1,2022-07-30 10:00:00
2042231,7852.y1001.60-15-d12-1.31.I,10:00:33,10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1,2022-07-30 10:00:00
2042232,7852.y1001.60-15-d12-1.31.I,10:02:03,10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1,2022-07-30 10:00:00
2042233,7852.y1001.60-15-d12-1.31.I,10:03:06,10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1,2022-07-30 10:00:00
2042234,7852.y1001.60-15-d12-1.31.I,10:05:00,10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1,2022-07-30 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...
2042293,7852.y1001.60-15-d12-1.31.I,11:15:55,11:15:55,8220DB006315,67,Clongriffin,0,0,21482.26,60-15-d12-1,2022-07-30 10:00:00
2042294,7852.y1001.60-15-d12-1.31.I,11:17:12,11:17:12,8220DB006316,68,Clongriffin,0,0,21964.97,60-15-d12-1,2022-07-30 10:00:00
2042295,7852.y1001.60-15-d12-1.31.I,11:18:12,11:18:12,8220DB007236,69,Clongriffin,0,0,22141.03,60-15-d12-1,2022-07-30 10:00:00
2042296,7852.y1001.60-15-d12-1.31.I,11:19:05,11:19:05,8220DB007245,70,Clongriffin,0,0,22308.07,60-15-d12-1,2022-07-30 10:00:00


In [519]:
df_stoptimes_merged[(df_stoptimes_merged['stop_id']!=df_stoptimes_merged['stop_id'])&(df_stoptimes_merged['stop_id'].notna())]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate,Arr_delay,Dep_delay,LineId


In [520]:
df_stoptimes_merged[(df_stoptimes_merged['stop_id']==df_stoptimes_merged['stop_id'])&(df_stoptimes_merged['stop_id'].notna())]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate,Arr_delay,Dep_delay,LineId
0,7852.y1001.60-15-d12-1.31.I,10:00:00,10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
1,7852.y1001.60-15-d12-1.31.I,10:00:33,10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
2,7852.y1001.60-15-d12-1.31.I,10:02:03,10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
3,7852.y1001.60-15-d12-1.31.I,10:03:06,10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
4,7852.y1001.60-15-d12-1.31.I,10:05:00,10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335860,1154.2.60-15-b12-1.32.O,16:20:46,16:20:46,8230DB004869,66,Ballycullen Road,0,0,20871.25,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15
335861,1154.2.60-15-b12-1.32.O,16:01:07,16:01:07,8230DB004870,67,Ballycullen Road,0,0,21682.89,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15
335862,1154.2.60-15-b12-1.32.O,16:01:37,16:01:37,8230DB003007,68,Ballycullen Road,0,0,21818.96,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15
335863,1154.2.60-15-b12-1.32.O,16:03:14,16:03:14,8230DB006283,69,Ballycullen Road,0,0,22252.87,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15


In [521]:
df_stoptimes_merged['StartDate']=df_stoptimes_merged['StartDate'].astype('datetime64')

In [522]:
df_stoptimes_merged['date_of_service'] = df_stoptimes_merged['StartDate'].dt.strftime("%Y-%m-%d")

In [523]:
df_stoptimes_merged['arrival_time'] = df_stoptimes_merged['date_of_service'] + str(' ') + df_stoptimes_merged['arrival_time']
df_stoptimes_merged['departure_time'] = df_stoptimes_merged['date_of_service'] + str(' ') + df_stoptimes_merged['departure_time']
df_stoptimes_merged=df_stoptimes_merged.drop(columns=['date_of_service'])

In [524]:
#convert the data type of time into datetime
df_stoptimes_merged['arrival_time'] = df_stoptimes_merged['arrival_time'].astype('datetime64')
df_stoptimes_merged['departure_time'] = df_stoptimes_merged['departure_time'].astype('datetime64')

In [527]:
df_stoptimes_merged.dtypes

trip_id                        object
arrival_time           datetime64[ns]
departure_time         datetime64[ns]
stop_id                        object
stop_sequence                   int64
stop_headsign                  object
pickup_type                     int64
drop_off_type                   int64
shape_dist_traveled           float64
trip_id_new                    object
StartDate              datetime64[ns]
Arr_delay                     float64
Dep_delay                     float64
LineId                         object
dtype: object

In [513]:
#calcuate the actual depart time and actual arrive time
df_stoptimes_merged['actual_arrival_time'] = 0
df_stoptimes_merged['actual_departure_time'] = 0
for i in range(len(df_stoptimes_merged)):
    df_stoptimes_merged['actual_arrival_time'][i] = df_stoptimes_merged['arrival_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Arr_delay'][i])
    df_stoptimes_merged['actual_departure_time'][i] = df_stoptimes_merged['departure_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Dep_delay'][i])
    print(i)

<ipython-input-513-528f7c9f4585>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stoptimes_merged['actual_arrival_time'][i] = df_stoptimes_merged['arrival_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Arr_delay'][i])
<ipython-input-513-528f7c9f4585>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stoptimes_merged['actual_departure_time'][i] = df_stoptimes_merged['departure_time'][i] + pd.Timedelta(seconds=df_stoptimes_merged['Dep_delay'][i])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [422]:
df_stoptimes_merged

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,StartDate,Arr_delay,Dep_delay,LineId,actual_arrival_time,actual_departure_time
0,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:00:00,2022-07-24 10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15,2022-07-24 10:00:00,2022-07-24 10:00:00
1,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:00:33,2022-07-24 10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15,2022-07-24 10:00:33,2022-07-24 10:00:33
2,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:02:03,2022-07-24 10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15,2022-07-24 10:02:03,2022-07-24 10:02:03
3,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:03:06,2022-07-24 10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15,2022-07-24 10:03:06,2022-07-24 10:03:06
4,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:05:00,2022-07-24 10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1,2022-07-24 10:00:00,0.0,0.0,15,2022-07-24 10:05:00,2022-07-24 10:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335860,1154.2.60-15-b12-1.32.O,2022-08-03 16:20:46,2022-08-03 16:20:46,8230DB004869,66,Ballycullen Road,0,0,20871.25,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15,2022-08-03 16:20:46,2022-08-03 16:20:46
335861,1154.2.60-15-b12-1.32.O,2022-08-03 16:01:07,2022-08-03 16:01:07,8230DB004870,67,Ballycullen Road,0,0,21682.89,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15,2022-08-03 16:01:07,2022-08-03 16:01:07
335862,1154.2.60-15-b12-1.32.O,2022-08-03 16:01:37,2022-08-03 16:01:37,8230DB003007,68,Ballycullen Road,0,0,21818.96,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15,2022-08-03 16:01:37,2022-08-03 16:01:37
335863,1154.2.60-15-b12-1.32.O,2022-08-03 16:03:14,2022-08-03 16:03:14,8230DB006283,69,Ballycullen Road,0,0,22252.87,60-15-b12-1,2022-08-03 14:50:00,0.0,0.0,15,2022-08-03 16:03:14,2022-08-03 16:03:14


In [423]:
df_stoptimes_merged['stop_headsign'].value_counts()

 Clongriffin         176528
 Ballycullen Road    159337
Name: stop_headsign, dtype: int64

In [425]:
#convert the data type of time into datetime
df_stoptimes_merged['actual_arrival_time'] = df_stoptimes_merged['actual_arrival_time'].astype('datetime64')
df_stoptimes_merged['actual_departure_time'] = df_stoptimes_merged['actual_departure_time'].astype('datetime64')

In [426]:
df_stoptimes_merged.dtypes

trip_id                          object
arrival_time             datetime64[ns]
departure_time           datetime64[ns]
stop_id                          object
stop_sequence                     int64
stop_headsign                    object
pickup_type                       int64
drop_off_type                     int64
shape_dist_traveled             float64
trip_id_new                      object
StartDate                datetime64[ns]
Arr_delay                       float64
Dep_delay                       float64
LineId                           object
actual_arrival_time      datetime64[ns]
actual_departure_time    datetime64[ns]
dtype: object

In [467]:
#Calculate the accumulated travel time
df_stoptimes_merged['acc_travel_time'] = (df_stoptimes_merged['actual_arrival_time'] - df_stoptimes_merged['StartDate'])
df_stoptimes_merged['acc_travel_time'] = df_stoptimes_merged['acc_travel_time'].dt.total_seconds()


In [437]:
df_stoptimes_merged['hour_of_day']=df_stoptimes_merged['actual_arrival_time'].dt.hour
df_stoptimes_merged['day_of_week']=df_stoptimes_merged['actual_arrival_time'].dt.dayofweek
df_stoptimes_merged['month_of_year']=df_stoptimes_merged['actual_arrival_time'].dt.month

In [482]:
df_stoptimes_merged['date']=df_stoptimes_merged['actual_arrival_time'].astype('datetime64[h]')

In [488]:
df_stoptimes_merged=pd.merge(df_stoptimes_merged,weather[['date','temp','rain']],how="left",on=['date'],)

In [489]:
df_stoptimes_merged

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,trip_id_new,...,actual_departure_time,acc_travel_time(s),acc_travel_time,hour_of_day,day_of_week,month_of_year,actual_arrival_time_copy,date,temp,rain
0,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:00:00,2022-07-24 10:00:00,8230DB006282,1,Clongriffin,0,0,0.00,60-15-d12-1,...,2022-07-24 10:00:00,0 days 00:00:00,0.0,10,6,7,2022-07-24 10:00:00,2022-07-24 10:00:00,22.3,0.0
1,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:00:33,2022-07-24 10:00:33,8230DB006335,2,Clongriffin,0,0,121.51,60-15-d12-1,...,2022-07-24 10:00:33,0 days 00:00:33,33.0,10,6,7,2022-07-24 10:00:00,2022-07-24 10:00:00,22.3,0.0
2,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:02:03,2022-07-24 10:02:03,8230DB006326,3,Clongriffin,0,0,451.15,60-15-d12-1,...,2022-07-24 10:02:03,0 days 00:02:03,123.0,10,6,7,2022-07-24 10:00:00,2022-07-24 10:00:00,22.3,0.0
3,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:03:06,2022-07-24 10:03:06,8230DB004886,4,Clongriffin,0,0,691.38,60-15-d12-1,...,2022-07-24 10:03:06,0 days 00:03:06,186.0,10,6,7,2022-07-24 10:00:00,2022-07-24 10:00:00,22.3,0.0
4,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:05:00,2022-07-24 10:05:00,8230DB004887,5,Clongriffin,0,0,1529.38,60-15-d12-1,...,2022-07-24 10:05:00,0 days 00:05:00,300.0,10,6,7,2022-07-24 10:00:00,2022-07-24 10:00:00,22.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335860,1154.2.60-15-b12-1.32.O,2022-08-03 16:20:46,2022-08-03 16:20:46,8230DB004869,66,Ballycullen Road,0,0,20871.25,60-15-b12-1,...,2022-08-03 16:20:46,0 days 01:30:46,5446.0,16,2,8,2022-08-03 16:00:00,2022-08-03 16:00:00,NaN,NaN
335861,1154.2.60-15-b12-1.32.O,2022-08-03 16:01:07,2022-08-03 16:01:07,8230DB004870,67,Ballycullen Road,0,0,21682.89,60-15-b12-1,...,2022-08-03 16:01:07,0 days 01:11:07,4267.0,16,2,8,2022-08-03 16:00:00,2022-08-03 16:00:00,NaN,NaN
335862,1154.2.60-15-b12-1.32.O,2022-08-03 16:01:37,2022-08-03 16:01:37,8230DB003007,68,Ballycullen Road,0,0,21818.96,60-15-b12-1,...,2022-08-03 16:01:37,0 days 01:11:37,4297.0,16,2,8,2022-08-03 16:00:00,2022-08-03 16:00:00,NaN,NaN
335863,1154.2.60-15-b12-1.32.O,2022-08-03 16:03:14,2022-08-03 16:03:14,8230DB006283,69,Ballycullen Road,0,0,22252.87,60-15-b12-1,...,2022-08-03 16:03:14,0 days 01:13:14,4394.0,16,2,8,2022-08-03 16:00:00,2022-08-03 16:00:00,NaN,NaN


In [487]:
weather = pd.read_csv('hly175.csv')
weather['date']=weather['date'].astype('datetime64')

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2,4,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [490]:
#Clean up the columns
df_stoptimes_merged_cleaned = df_stoptimes_merged[['trip_id','arrival_time','stop_id','stop_sequence','stop_headsign','trip_id_new','StartDate','LineId','actual_arrival_time','actual_departure_time','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']]

In [491]:
df_stoptimes_merged_cleaned

,trip_id,arrival_time,stop_id,stop_sequence,stop_headsign,trip_id_new,StartDate,LineId,actual_arrival_time,actual_departure_time,acc_travel_time,hour_of_day,day_of_week,month_of_year,temp,rain
0,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:00:00,8230DB006282,1,Clongriffin,60-15-d12-1,2022-07-24 10:00:00,15,2022-07-24 10:00:00,2022-07-24 10:00:00,0.0,10,6,7,22.3,0.0
1,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:00:33,8230DB006335,2,Clongriffin,60-15-d12-1,2022-07-24 10:00:00,15,2022-07-24 10:00:33,2022-07-24 10:00:33,33.0,10,6,7,22.3,0.0
2,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:02:03,8230DB006326,3,Clongriffin,60-15-d12-1,2022-07-24 10:00:00,15,2022-07-24 10:02:03,2022-07-24 10:02:03,123.0,10,6,7,22.3,0.0
3,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:03:06,8230DB004886,4,Clongriffin,60-15-d12-1,2022-07-24 10:00:00,15,2022-07-24 10:03:06,2022-07-24 10:03:06,186.0,10,6,7,22.3,0.0
4,7852.y1001.60-15-d12-1.31.I,2022-07-24 10:05:00,8230DB004887,5,Clongriffin,60-15-d12-1,2022-07-24 10:00:00,15,2022-07-24 10:05:00,2022-07-24 10:05:00,300.0,10,6,7,22.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335860,1154.2.60-15-b12-1.32.O,2022-08-03 16:20:46,8230DB004869,66,Ballycullen Road,60-15-b12-1,2022-08-03 14:50:00,15,2022-08-03 16:20:46,2022-08-03 16:20:46,5446.0,16,2,8,NaN,NaN
335861,1154.2.60-15-b12-1.32.O,2022-08-03 16:01:07,8230DB004870,67,Ballycullen Road,60-15-b12-1,2022-08-03 14:50:00,15,2022-08-03 16:01:07,2022-08-03 16:01:07,4267.0,16,2,8,NaN,NaN
335862,1154.2.60-15-b12-1.32.O,2022-08-03 16:01:37,8230DB003007,68,Ballycullen Road,60-15-b12-1,2022-08-03 14:50:00,15,2022-08-03 16:01:37,2022-08-03 16:01:37,4297.0,16,2,8,NaN,NaN
335863,1154.2.60-15-b12-1.32.O,2022-08-03 16:03:14,8230DB006283,69,Ballycullen Road,60-15-b12-1,2022-08-03 14:50:00,15,2022-08-03 16:03:14,2022-08-03 16:03:14,4394.0,16,2,8,NaN,NaN


In [492]:
df_stoptimes_merged_cleaned.to_csv('df_stoptimes_route15_merged_cleaned.csv',index=False)

In [56]:
df_stoptimes_merged_cleaned = pd.read_csv('df_stoptimes_route15_merged_cleaned.csv')

In [57]:
df_stoptimes_merged_cleaned.dtypes

trip_id                   object
arrival_time              object
stop_id                   object
stop_sequence              int64
stop_headsign             object
trip_id_new               object
StartDate                 object
LineId                     int64
actual_arrival_time       object
actual_departure_time     object
acc_travel_time          float64
hour_of_day                int64
day_of_week                int64
month_of_year              int64
temp                     float64
rain                     float64
dtype: object

In [63]:
df_stoptimes_merged_cleaned['temp'].fillna(0, inplace=True)
df_stoptimes_merged_cleaned['rain'].fillna(0, inplace=True)

#Select the features we need for modeling and Copy the original version of dataset
df_features_for_ML = df_stoptimes_merged_cleaned[['stop_sequence','stop_headsign','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']]
df_features_for_ML['stop_headsign'].astype('category')
# set up dummies features
df_features_for_ML = pd.get_dummies(df_features_for_ML, drop_first=False)


#Setting up the train/test split
# y is the target
y = df_features_for_ML["acc_travel_time"]
# X is everything else
X = df_features_for_ML.drop(["acc_travel_time"],1)
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


# Train aka fit, a model using all continuous and categorical features.
linreg = LinearRegression().fit(X_train, y_train)

#Predict the test dataset
linreg_predictions_test = linreg.predict(X_test)

In [64]:
df_features_for_ML.dtypes

stop_sequence                        int64
acc_travel_time                    float64
hour_of_day                          int64
day_of_week                          int64
month_of_year                        int64
temp                               float64
rain                               float64
stop_headsign_ Ballycullen Road      uint8
stop_headsign_ Clongriffin           uint8
dtype: object

In [75]:
pickle.dump(linreg, open('linreg_15_fortesting.pkl', 'wb'))

In [66]:
#This function is used repeatedly to compute all metrics
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [67]:
print("\nPredictions with linear regression: \n")
actual_vs_predicted_linreg = pd.concat([y_test, pd.DataFrame(linreg_predictions_test, columns=['Predicted'], index=y_test.index)], axis=1)
print(actual_vs_predicted_linreg)


Predictions with linear regression: 

        acc_travel_time    Predicted
54463            4629.0  4398.907352
278622           3306.0  3352.420465
233303           1257.0  1142.966857
109723           4622.0  4328.129967
258693           1198.0  1726.850293
...                 ...          ...
84406            1245.0  1929.484470
276342           3726.0  3230.965479
240183            854.0   779.562848
305246            500.0   431.809092
206388           1631.0  2054.781565

[100760 rows x 2 columns]


In [68]:
printMetrics(y_test,linreg_predictions_test)


MAE:  506.6449875496814
RMSE:  706.1725630397968
R2:  0.810641802626981


In [ ]:
df_stoptimes

In [ ]:
#Try it with random forest

In [543]:
df_stoptimes_merged_cleaned =pd.read_csv('df_stoptimes_route15_merged_cleaned.csv')

In [545]:
df_stoptimes_merged_cleaned['temp'].fillna(0, inplace=True)
df_stoptimes_merged_cleaned['rain'].fillna(0, inplace=True)

In [546]:
#Select the features we need for modeling and Copy the original version of dataset
df_features_for_ML = df_stoptimes_merged_cleaned[['stop_sequence','stop_headsign','acc_travel_time','hour_of_day','day_of_week','month_of_year','temp','rain']]
df_features_for_ML['stop_headsign'].astype('category')

# set up dummies features
df_features_for_ML = pd.get_dummies(df_features_for_ML, drop_first=True)


#Setting up the train/test split
# y is the target
y = df_features_for_ML["acc_travel_time"]
# X is everything else
X = df_features_for_ML.drop(["acc_travel_time"],1)
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,  test_size=0.3)


# Train RF with 10 trees
rfc = RandomForestRegressor(n_estimators=10, max_features='auto', oob_score=True, random_state=1).fit(X_train, y_train)

#Predict the test dataset
rfc_predictions_test = rfc.predict(X_test)

/Users/oujunhao/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:833: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [547]:
printMetrics(y_test,rfc_predictions_test)


MAE:  334.8427090012246
RMSE:  513.7153115937551
R2:  0.8997908331264771


In [97]:
#Load the model
test_model = pickle.load(open('ModelResult_RandomForest_GTFS/PickleFiles_722.pkl', 'rb'))

In [98]:
dictionary = {'stop_sequence':15,'hour_of_day':8,'day_of_week':5,'month_of_year':8,'temp':23,'rain':0,'stop_headsign_ Ballycullen':1,'stop_headsign_ Clongriffin':0}

In [99]:
var = pd.DataFrame([dictionary])
var

,stop_sequence,hour_of_day,day_of_week,month_of_year,temp,rain,stop_headsign_ Ballycullen,stop_headsign_ Clongriffin
0,15,8,5,8,23,0,1,0


In [100]:
test_model.predict(var)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 75 is different from 8)